In [441]:
import aop
import aop.api
import json
import time

import pymysql
import pandas as pd
import numpy as np
import datetime
from urllib.parse import unquote

#google sheet 專用   #pip install google-api-python-client
import string
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [442]:
# 時間日期設定
today=(datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d") 
# 存檔案路徑
downloadpath='//navdi/DEM_UserData/btorin/Desktop/download_csv/'
# 设置网关域名
aop.set_default_server('gateway.open.umeng.com')
# 设置apiKey和apiSecurity
aop.set_default_appinfo(9048862, "76wqCO5k9U")
# appkey_channel 
appkey_h5_new='62b2ca6788ccdf4b7ea5fb49'
appkey_app_new='62d8f1cd88ccdf4b7ed9b9f3'

# H5 DATA

In [443]:
req = aop.api.UmengUminiGetOverviewRequest()
resp_count = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="visitTimes", pageIndex=1, pageSize=1)
resp_active = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="activeUser", pageIndex=1, pageSize=1)
resp_new = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="newUser", pageIndex=1, pageSize=1)
#轉換
resp_count=resp_count['data']['data'][0]  ; resp_active=resp_active['data']['data'][0]  ; resp_new=resp_new['data']['data'][0]
resp_count["value"] = resp_count.pop("visitTimes")  ;resp_active["value"] = resp_active.pop("activeUser")  ; resp_new["value"] = resp_new.pop("newUser")
resp_total=[resp_count,resp_active,resp_new]
resp_total=pd.DataFrame(resp_total)
resp_total['item']=['启动次数','活跃会员','新增用户']
resp_total['device']='H5'
dd_data_h5=resp_total[['dateTime','device','item','value']]
dd_data_h5.columns=['date','device','item','value']

In [444]:
#外層事件
req1 = aop.api.UmengUminiGetAllPropertyValueCountRequest()
resp1 = req1.get_response(None, timeUnit="day", fromDate=today, toDate=today, propertyName="category", pageIndex=1, pageSize=100, eventName="click", dataSourceId=appkey_h5_new) 
resp1_evevt=pd.DataFrame(resp1['data']['data'])
resp1_evevt['date']=today ;resp1_evevt['device']='H5'
resp1_evevt=resp1_evevt[['date','device','propertyValue','count','countRatio']]
#內層事件
req2 = aop.api.UmengUminiGetAllPropertyValueCountRequest()
resp2 = req2.get_response(None, timeUnit="day", fromDate=today, toDate=today, propertyName="label", pageIndex=1, pageSize=100, eventName="click", dataSourceId=appkey_h5_new)  
resp2_evevt=pd.DataFrame(resp2['data']['data'])
resp2_evevt['date']=today ;resp2_evevt['device']='H5'
resp2_evevt=resp2_evevt[['date','device','propertyValue','count','countRatio']]
#搜尋事件
req3 = aop.api.UmengUminiGetAllPropertyValueCountRequest()
resp3 = req3.get_response(None, timeUnit="day", fromDate=today, toDate=today, propertyName="value", pageIndex=1, pageSize=100, eventName="search", dataSourceId=appkey_h5_new)  
resp3_evevt=pd.DataFrame(resp3['data']['data'])
resp3_evevt['date']=today ;resp3_evevt['device']='H5'
resp3_evevt=resp3_evevt[['date','device','propertyValue','count','countRatio']]
#影片點擊
req4 = aop.api.UmengUminiGetAllPropertyValueCountRequest()
resp4 = req4.get_response(None, timeUnit="day", fromDate=today, toDate=today, propertyName="from_video", pageIndex=1, pageSize=200, eventName="watch", dataSourceId=appkey_h5_new)  
resp4_evevt=pd.DataFrame(resp4['data']['data'])
resp4_evevt['date']=today ;resp4_evevt['device']='H5'
resp4_evevt=resp4_evevt[['date','device','propertyValue','count','countRatio']]

# APP DATA

In [445]:
req = aop.api.UmengUappGetLaunchesByChannelOrVersionRequest()
resp_count = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, periodType="daily", channels="", versions="")
# 发起Api请求 活躍會員
req = aop.api.UmengUappGetActiveUsersByChannelOrVersionRequest()
resp_active = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, periodType="daily", channels="", versions="")
# 发起Api请求 新增用戶數
req = aop.api.UmengUappGetNewUsersByChannelOrVersionRequest()
resp_new = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, periodType="daily", channels="", versions="")
resp_total=resp_count['launchInfo']+resp_active['activeUserInfo']+resp_new['newUserInfo']
resp_total=pd.DataFrame(resp_total)
resp_total['item']=['启动次数','活跃会员','新增用户']
resp_total['device']='APP'
dd_data_app=resp_total[['date','device','item','value']]

In [446]:
# 外層事件
req = aop.api.UmengUappEventParamGetValueListRequest()
resp = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, eventName="click", eventParamName="category")
test_cc1=pd.DataFrame(resp['paramInfos'])
test_cc1['name_c'] = test_cc1.apply(lambda row : unquote(row['name']), axis = 1)
test_cc1.columns=['name','count','countRatio','propertyValue']
test_cc1['date']=today ;test_cc1['device']='APP'
test_cc1=test_cc1[['date','device','propertyValue','count','countRatio']]
test_cc1=test_cc1.head(100)
# 內層事件
req = aop.api.UmengUappEventParamGetValueListRequest()
resp = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, eventName="click", eventParamName="label")
test_cc2=pd.DataFrame(resp['paramInfos'])
test_cc2['name_c'] = test_cc2.apply(lambda row : unquote(row['name']), axis = 1)
test_cc2.columns=['name','count','countRatio','propertyValue']
test_cc2['date']=today ;test_cc2['device']='APP'
test_cc2=test_cc2[['date','device','propertyValue','count','countRatio']]
test_cc2=test_cc2.head(100)
# 搜尋詞彙
req = aop.api.UmengUappEventParamGetValueListRequest()
resp = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, eventName="search", eventParamName="value")
test_cc3=pd.DataFrame(resp['paramInfos'])
test_cc3['name_c'] = test_cc3.apply(lambda row : unquote(row['name']), axis = 1)
test_cc3.columns=['name','count','countRatio','propertyValue']
test_cc3['date']=today ;test_cc3['device']='APP'
test_cc3=test_cc3[['date','device','propertyValue','count','countRatio']]
test_cc3=test_cc3.head(100)
#觀看影片
req = aop.api.UmengUappEventParamGetValueListRequest()
resp = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, eventName="watch", eventParamName="from_video")
test_cc4=pd.DataFrame(resp['paramInfos'])
test_cc4['name_c'] = test_cc4.apply(lambda row : unquote(row['name']), axis = 1)
test_cc4.columns=['name','count','countRatio','propertyValue']
test_cc4['date']=today ;test_cc4['device']='APP'
test_cc4=test_cc4[['date','device','propertyValue','count','countRatio']]
test_cc4=test_cc4.head(200)

# Merge h5 app

In [447]:
#合併流量
dd_data=dd_data_h5.append(dd_data_app)
#合併事件
dd_event1=resp1_evevt.append(test_cc1)
dd_event2=resp2_evevt.append(test_cc2)
dd_event3=resp3_evevt.append(test_cc3)
dd_event4=resp4_evevt.append(test_cc4)

#  寫入雲端表格

In [448]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
      'https://www.googleapis.com/auth/drive',
      'https://www.googleapis.com/auth/spreadsheets']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()

#寫入 data 進入 google sheet #美東時間
def insert_data(value_range_body,sheetname,colname1,colname2,col1,col2) :
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request= sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,valueInputOption='USER_ENTERED',range=SAMPLE_RANGE_NAME , body=value_range_body).execute()
def google_sheet_insert_data(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A30000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','D',insert_row,insert_row+50)
    
def google_sheet_insert_data_event(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A70000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','E',insert_row,insert_row+500)

In [449]:
#流量數據
SAMPLE_SPREADSHEET_ID = '14OR2-WaGLyZfFpMWbxs_g7sureaZYxCb0XKKMmpMZpU'
google_sheet_insert_data('會員數據',dd_data,SAMPLE_SPREADSHEET_ID)
#外層事件數據
google_sheet_insert_data_event('外层事件',dd_event1,SAMPLE_SPREADSHEET_ID)
#內層事件數據
google_sheet_insert_data_event('内层事件',dd_event2,SAMPLE_SPREADSHEET_ID)
#搜尋事件數據
google_sheet_insert_data_event('搜寻栏',dd_event3,SAMPLE_SPREADSHEET_ID)
#搜尋事件數據
google_sheet_insert_data_event('影片事件',dd_event4,SAMPLE_SPREADSHEET_ID)

In [ ]:
#req = aop.api.UmengUappEventListRequest()
#resp = req.get_response(None, appkey=appkey_app_new, startDate=today, endDate=today, perPage=100, page=1, version="")
#resp
##req = aop.api.UmengUappEventParamListRequest()
##resp = req.get_response(None, startDate=today, endDate=today, eventId="62df4ce6475da914ef9bbeaf", appkey=appkey_app_new)
##resp